# Give Me Some Credit

## Get the train dataset

In [ ]:
import pandas as pd
GMSC_train_data = pd.read_csv('sources/cs-training.csv', index_col=0)
GMSC_train_data.describe()

In [ ]:
Y_train = GMSC_train_data['SeriousDlqin2yrs']
GMSC_train_data.drop(columns=['SeriousDlqin2yrs'], inplace=True)
X_train = GMSC_train_data

## Preprocess the data

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="mean")),
        ('std_scaler', RobustScaler()),
    ])

X_train_std = num_pipeline.fit_transform(X_train)

## Find bests hyperparameters for XGBClassifier with GridSearchCV 

In [ ]:
import xgboost as xgboost
from sklearn.model_selection import StratifiedKFold, GridSearchCV

xgb = xgboost.XGBClassifier(
    nthread = 8,
    objective ='binary:logistic',
    tree_method = 'gpu_hist',
    gpu_id = 0,
#     predictor = 'cpu_predictor'
)

skf = StratifiedKFold(
    n_splits = 5,
    shuffle = True,
    random_state = 42
)

In [ ]:
params = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

grid_search = GridSearchCV(
    xgb,
    param_grid=params,
    scoring='roc_auc',
    n_jobs=8,
    cv=skf.split(X_train_std, Y_train),
    verbose=3,
)

grid_search.fit(X_train_std, Y_train)

In [ ]:
print(f'Best score : {grid_search.best_score_}')
print(f'Best estimator: {grid_search.best_estimator_}')
print(f'Best hyperparameters: {grid_search.best_params_}')

## Get the test dataset

In [ ]:
GMSC_test_data = pd.read_csv('sources/cs-test.csv')
id_test = GMSC_test_data['Unnamed: 0']
GMSC_test_data.drop(columns=['SeriousDlqin2yrs'], inplace=True)
GMSC_test_data.drop(columns=['Unnamed: 0'], inplace=True)
GMSC_test_data.describe()

## Make a batch prediction on the test dataset

In [ ]:
X_test_std = num_pipeline.transform(GMSC_test_data)
y_pred = grid_search.predict_proba(X_test_std)
kaggle_df = pd.DataFrame(data={'id':id_test, 'Probability':y_pred[:,1]})
kaggle_df.to_csv('submission-kaggle.csv', index=False)